In [1]:
import pandas as pd
import numpy as np
from plotnine import *

%matplotlib inline

In [2]:
data = pd.read_csv("../../data/processed/expensas_full_processed_vis.csv",
                  dtype = {
                                "expensa_id": "object",
                                "unidad_id": "object",
                                "consorcio_id": "object",
                                "expensa_mes":"object",
                            },
                  parse_dates = ["expensa_fecha", "expensa_mes_pago", "expensa_mes_pago_anterior"])

In [3]:
data2 = data.loc[:,["expensa_id", "consorcio_id", "unidad_id"]].groupby(["consorcio_id", "unidad_id"]).count().reset_index()[["consorcio_id", "unidad_id"]]

data2.consorcio_id = data2.consorcio_id.map(int)
data2.unidad_id = data2.unidad_id.map(int)

data2 = data2.sort_values(["consorcio_id","unidad_id"])
data2["unidad_id_bis"] = range(0, len(data2))

data2.consorcio_id = data2.consorcio_id.map(str)
data2.unidad_id = data2.unidad_id.map(str)

In [4]:
data2 = pd.merge(data,
                 data2,
                 left_on=["consorcio_id", "unidad_id"],
                 right_on=["consorcio_id", "unidad_id"],
                 how="left")

In [5]:
data2.shape

(222968, 20)

In [6]:
data.shape

(222968, 19)

In [7]:
data2.unidad_id_bis.max()

14513

In [9]:
def month_diff(a, b):
    return 12 * (a.dt.year - b.dt.year) + (a.dt.month - b.dt.month)


In [45]:
month_diff(consorcios_rect.y_max,consorcios_rect.y_min)[0]

58

In [13]:
min_id = 13157
for max_id in range(13400,14513,500):
    consorcios_plot = data2.loc[(data2.unidad_id_bis >=  min_id) & (data2.unidad_id_bis <=  max_id), "consorcio_id"].unique()
    data_plot = data2.loc[data2.consorcio_id.isin(consorcios_plot)]
    max_id = data_plot.unidad_id_bis.max()
    print("desde {}, hasta {}".format(min_id, max_id))
    consorcios_rect = data2.loc[data2.consorcio_id.isin(consorcios_plot), ["consorcio_id", "consorcio_nombre","unidad_id_bis"]].groupby(["consorcio_id", "consorcio_nombre"]).min().reset_index()
    consorcios_rect.columns = ["consorcio","nombre","x_min"]
    consorcios_rect.x_min = consorcios_rect.x_min-0.5
    consorcios_rect2 = data2.loc[data2.consorcio_id.isin(consorcios_plot), ["consorcio_id", "consorcio_nombre","unidad_id_bis"]].groupby(["consorcio_id", "consorcio_nombre"]).max().reset_index()
    consorcios_rect2.columns = ["consorcio","nombre","x_max"]
    consorcios_rect2.x_max = consorcios_rect2.x_max+0.5

    consorcios_rect = pd.merge(consorcios_rect,
                               consorcios_rect2,
                               left_on=["consorcio", "nombre"],
                               right_on=["consorcio","nombre"])

    consorcios_rect["y_min"]= data_plot.expensa_mes_pago.min()
    consorcios_rect["y_max"]= data_plot.expensa_mes_pago.max()
    consorcios_rect = consorcios_rect.sort_values("x_min").reset_index().drop(columns="index")
    consorcios_rect["x_name"] = (consorcios_rect.x_min + consorcios_rect.x_max)/2
    consorcios_rect["y_name"] = pd.to_datetime("2019-02-01")
    consorcios_rect.loc[::2, "y_name"] = pd.to_datetime("2019-06-01")

    width = min(0.08 * (max_id - min_id), 50)
    height = 0.08 * month_diff(consorcios_rect.y_max,consorcios_rect.y_min)[0] 
    height = height if height > 0 else 2
    
    (ggplot(data_plot)
     #+ geom_rect(aes(xmin="x_min", xmax = "x_max",  ymin="y_min", ymax="y_max", fill = "color"), alpha=0.4, data = consorcios_rect)
     + geom_point(aes(x="unidad_id_bis", y="expensa_mes_pago",color ="factor(pago_metodo)"), size=1.5, shape="s")
     + scale_color_manual(["#7c94ff","#567c27","#ff1e1e", "#b5af05", "#9b04c9"], name="Paga Tarde") 
     + geom_vline(aes(xintercept="x_max"), size=1, color = "black", data=consorcios_rect)
     + geom_text(aes(x="x_name", y="y_name", label="nombre"), data=consorcios_rect)
     + theme(
         panel_background = element_rect(fill = "white"),
         axis_text_x = element_text(angle = 90, hjust = 1)
     )
    ).save(filename = 'historial_grafico_{}_{}.png'.format(min_id, max_id), height=height, width=width, units = 'in', dpi=1000, limitsize=False)
    
    min_id = max_id + 1 

desde 13157, hasta 13445


C:\Anaconda3\lib\site-packages\plotnine\ggplot.py:706: UserWarning: Saving 23.04 x 2 in image.
  from_inches(height, units), units))
C:\Anaconda3\lib\site-packages\plotnine\ggplot.py:707: UserWarning: Filename: historial_grafico_13157_13445.png
  warn('Filename: {}'.format(filename))


desde 13446, hasta 13997


C:\Anaconda3\lib\site-packages\plotnine\ggplot.py:706: UserWarning: Saving 44.08 x 2 in image.
  from_inches(height, units), units))
C:\Anaconda3\lib\site-packages\plotnine\ggplot.py:707: UserWarning: Filename: historial_grafico_13446_13997.png
  warn('Filename: {}'.format(filename))


desde 13998, hasta 14405


C:\Anaconda3\lib\site-packages\plotnine\ggplot.py:706: UserWarning: Saving 32.56 x 2 in image.
  from_inches(height, units), units))
C:\Anaconda3\lib\site-packages\plotnine\ggplot.py:707: UserWarning: Filename: historial_grafico_13998_14405.png
  warn('Filename: {}'.format(filename))


In [61]:
consorcios_rect2 = data2.loc[data2.consorcio_id.isin(consorcios_plot), ["consorcio_id","unidad_id_bis"]].groupby("consorcio_id").max().reset_index()
consorcios_rect2.columns = ["consorcio", "x_max"]
consorcios_rect2.x_max = consorcios_rect2.x_max+0.5

,consorcio,x_max
0,11,146.5
1,355,266.5
2,48,200.5
3,518,410.5
4,533,441.5
5,534,487.5
6,539,508.5
7,568,542.5
8,649,586.5
9,650,598.5
